# Yahoo Finance

In [8]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
import pytz


In [9]:
frequency = "1d"
start_time = "2012-01-01"
end_time = (datetime.now(pytz.timezone('US/Pacific')) + timedelta(days=1)).strftime('%Y-%m-%d')

1. **TLT (iShares 20+ Year Treasury Bond ETF)**: Invests in U.S. Treasury bonds with remaining maturities greater than 20 years, aiming to track the investment results of an index composed of U.S. Treasury bonds.

2. **IEF (iShares 7-10 Year Treasury Bond ETF)**: Seeks to track the investment results of an index composed of U.S. Treasury bonds with remaining maturities between 7 and 10 years.

3. **EZU (iShares MSCI Eurozone ETF)**: Provides exposure to large and mid-sized companies in Eurozone countries, tracking the MSCI EMU Index.

4. **CNYA (iShares MSCI China A ETF)**: Offers exposure to large and mid-cap Chinese equities in the A-shares market, tracking the MSCI China A Inclusion Index.

5. **EWJ (iShares MSCI Japan ETF)**: Tracks the investment results of an index composed of Japanese equities, representing large and mid-sized companies in Japan.

6. **IGLT.L (iShares Core UK Gilts UCITS ETF)**: Invests in UK government bonds (gilts), seeking to track the performance of an index composed of sterling-denominated UK government bonds.

7. **XBB.TO (iShares Core Canadian Universe Bond Index ETF)**: Offers exposure to Canadian government and corporate bonds, tracking the performance of the broad Canadian bond market.

8. **BOND.AX (PIMCO Australian Bond Index Fund)**: Provides diversified exposure to the Australian bond market, including government, semi-government, and corporate debt securities.

9. **INDA (iShares MSCI India ETF)**: Aims to track the investment results of an index composed of Indian equities, representing large and mid-sized companies in India.

10. **EWL (iShares MSCI Switzerland ETF)**: Tracks the investment results of an index composed of Swiss equities, representing the Swiss stock market.

11. **ERUS (iShares MSCI Russia ETF)**: Seeks to track the investment results of an index composed of Russian equities.

12. **EWZ (iShares MSCI Brazil ETF)**: Aims to track the investment results of an index composed of Brazilian equities, reflecting the performance of the Brazilian stock market.

13. **EWY (iShares MSCI South Korea ETF)**: Tracks the investment results of an index composed of South Korean equities, representing the South Korean stock market.

14. **ENZL (iShares MSCI New Zealand ETF)**: Seeks to track the investment results of an index composed of New Zealand equities.

15. **EWD (iShares MSCI Sweden ETF)**: Aims to track the investment results of an index composed of Swedish equities, representing the Swedish stock market.

16. **EWM (iShares MSCI Malaysia ETF)**: Seeks to track the investment results of an index composed of Malaysian equities, representing the Malaysian stock market.

In [10]:
# Existing tickers and additional ETF tickers
tickers = ["BTC-CAD"]

# Retrieve historical data
historical_data = {}
for ticker in tickers:
    historical_data[ticker] = yf.download(ticker, start=start_time, end=end_time, interval=frequency)[['High', 'Low']]

# Combine and weight data
combined_data = pd.DataFrame()
for ticker, data in historical_data.items():
    for col in data.columns:
        combined_column_name = f"{ticker}_{col}"  # Create a unique column name for each 'High' and 'Low'
        combined_data[combined_column_name] = data[col]

# Calculate the Global Liquidity Index as the sum of all (weighted and non-weighted) columns
combined_data.fillna(method='ffill', inplace=True)
# combined_data now contains the weighted Global Liquidity Index and individual ticker data


[*********************100%%**********************]  1 of 1 completed
C:\Users\falty\AppData\Local\Temp\ipykernel_17856\3142488560.py:17: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  combined_data.fillna(method='ffill', inplace=True)


In [11]:
#combined_data.dropna(inplace=True)
combined_data

,BTC-CAD_High,BTC-CAD_Low
Date,,
2014-09-17,513.446411,497.302246
2014-09-18,502.728729,452.183655
2014-09-19,468.607666,419.985870
2014-09-20,463.974731,427.350739
2014-09-21,452.060120,430.965698
...,...,...
2024-03-04,85708.109375,83437.976562
2024-03-05,93044.843750,84514.890625
2024-03-06,93812.804688,80644.625000


In [12]:
combined_data.to_parquet('yahoo-finance.parquet')